In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
req = requests.get('http://browse.gmarket.co.kr/list?category=300028561&s=8')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
## CSS Selector를 통해 html요소들을 찾아낸다.

In [3]:
##타이틀
li_list = []
brand_list = []
title_list = []
for li in soup.select('div[class=box__item-title]'):
    li_list.append(li.text.strip())

for li in li_list:
    brand,title=tuple(li.split("상품명 "))
    brand = brand.replace("브랜드명 ","").strip()
    title = title.strip()
    
    brand_list.append(brand)
    title_list.append(title)

##가격
price_list = []
for price in soup.select('div[class=box__price-seller]'):
    price = price.text.replace("할인적용금액 ","")
    price = price.replace("상품금액 ","")
    price = price.replace(",","")
    price = price.replace("원","")
    price_list.append(price.strip())

##구매수(04.10)
count_list = []
for link in soup.find_all("ul", class_="list__score"):
    li = link.find("li",class_="list-item list-item__pay-count")
    #print(li)
    count = li.text.replace("구매 ","")
    count = count.replace("건","")
    count = count.replace(",","")
    count_list.append(int(count.strip()))
    
##링크
link_list = []
for link in soup.find_all("div", class_="box__item-title"):
    li = link.find('a')['href']
    link_list.append(li)

from pandas import DataFrame

df1 =  DataFrame({"brand": brand_list, "title": title_list, "purchased_count":count_list, "price": price_list,
                 "url":link_list})


today_date=datetime.today().strftime("%Y%m%d")
df1.to_excel('gmarket_scrapping_'+today_date+'.xlsx', index=True)
df1.head()

,brand,title,purchased_count,price,url
0,애플공식셀러,Apple 정품 에어팟 프로 MWP22KH/A,15408,257530,http://item.gmarket.co.kr/Item?goodscode=17608...
1,갤럭시,SM-R180 갤럭시버즈 라이브 블루투스이어폰 AKG 블랙,1156,128700,http://item.gmarket.co.kr/Item?goodscode=18631...
2,갤럭시,SM-R180 갤럭시버즈 라이브 블루투스이어폰 AKG 화이트,607,128700,http://item.gmarket.co.kr/Item?goodscode=18631...
3,삼성전자공식셀러,갤럭시버즈 프로 무선 블루투스이어폰 SM-R190 블랙,214,239800,http://item.gmarket.co.kr/Item?goodscode=20301...
4,피스넷공식셀러,끊김없는 완전무선 블루투스이어폰 피스넷 프리고,39058,49900,http://item.gmarket.co.kr/Item?goodscode=11478...
